---
# Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cmath
import math
import copy
import time
#import multiprocessing as mp

ModuleNotFoundError: No module named 'numpy'

---
# Functions for the calculation of vertices and drawing mesh

In [ ]:
# Beltrami Coefficient on Z-dom.
def nu(z):
    return mu(cmath.exp(z))*cmath.exp(-2*1j*z.imag)

# Update a triangle.
def newW1(nup,z1,z2,z3,w1,w2,w3):
    p=((z1-z3)/(z1-z2)+nup*(z1.conjugate()-z3.conjugate())/(z1.conjugate()-z2.conjugate()))/(1+nup)
    p0, p1 = p.real, p.imag
    u1, v1 =  w1.real, w1.imag
    u2, v2 =  w2.real, w2.imag
    u3, v3 =  w3.real, w3.imag
    abspsq = p0*p0 + p1*p1
    delta = 2*(abspsq-p0+1)
    a0 = ( -(u1-2*u2+u3)-p0*(u1+u2-2*u3)-p1*(v1+v2-2*v3) )/delta
    a1 = ( -(v1-2*v2+v3)-p0*(v1+v2-2*v3)+p1*(u1+u2-2*u3) )/delta
    b0 = (  (u1+u3)-p0*(u2+u3)+p1*(v2-v3)+abspsq*(u1+u2) )/delta
    b1 = (  (v1+v3)-p0*(v2+v3)-p1*(u2-u3)+abspsq*(v1+v2) )/delta
    b = b0 + b1*1j
    a = a0 + a1*1j
    return b, a + b, a*p+b

# for drawing fig. This should be parallelized.
def drawmesh(w,N,M):
    plt.figure(figsize=(10*M/(M+N), 10*N/(M+N)), dpi=120)
    for j in jlist:
        plt.plot(w[j].real,w[j].imag,'b-', lw=0.5)
    for k in klist:
        wtemp = np.zeros((len(jlist)),dtype=np.complex)
        wtemp = np.array([w[j][k] for j in jlist])
        ccode = 'r-' if (k%2)==0 else 'g-'
        plt.plot(wtemp.real,wtemp.imag, ccode, lw=0.5)
        if k>0:
            wtemp = np.array([w[j][k-(j%2)] for j in jlist])
            plt.plot(wtemp.real,wtemp.imag, 'k-', lw=0.5)        
    plt.show()

def iteration(z,w,N,M,itnam):
    for s in range(itnam):
        print ("iteration #",s)
        
        # Phase 1 & 2, j=0; Imaginary axis
        for k in klist:
            if k==0:
                w[0][k]=0
            else:
                w1,w2,w3 = newW1(nu(z[0][k]),
                                    z[0][k],z[0][k+1] if k<N-1 else z[0][0]+2*math.pi*1j,z[-1][k],
                                    w[0][k],w[0][k+1] if k<N-1 else w[0][0]+2*math.pi*1j,w[-1][k])
                w[0][k]=w[0][k-1]+abs(w1-w2)*1j

        # Phase 1 & 2, j>0
        for j in [-j for j in range(M)]:
            for k in klist:
                # Option: Option: it might be better if these are rotated.
                w1t,w2t,w3t = newW1(nu(z[j-1][k]),
                                    z[j-1][k],z[j][k+1] if k<N-1 else z[j][0]+2*math.pi*1j,z[j-1][k+1] if k<N-1 else z[j-1][0]+2*math.pi*1j,
                                    w[j-1][k],w[j][k+1] if k<N-1 else w[j][0]+2*math.pi*1j,w[j-1][k+1] if k<N-1 else w[j-1][0]+2*math.pi*1j)
                w1b,w2b,w3b = newW1(nu(z[j-1][k-1]),
                                    z[j-1][k-1] if k>0 else z[j][N-1]-2*math.pi*1j,z[j][k],z[j-1][k+1] if k<N-1 else z[j-1][0]+2*math.pi*1j,
                                    w[j-1][k-1] if k>0 else w[j][N-1]-2*math.pi*1j,w[j][k],w[j-1][k+1] if k<N-1 else w[j-1][0]+2*math.pi*1j)
                w1t=w1t-w2t+w[j][k+1] if k<N-1 else w3t-w2t+w[j][0]+2*math.pi*1j
                w3b=w3b-w2b+w[j][k]
                w[j-1][k]=(w1t+w3b)/2

        # Phase 3
        for j in [-j for j in range(M)]:
            a=2*math.pi*1j/(w[j][N-1]-w[j][0]) # w[j][N-1] is should be replaced to w[j][N].
            b=math.pi*1j-a*w[j][N-1]
            nw00=a*w[0][0]+b
            for k in klist:
                w[j][k]=a*w[j][k]+b-nw00
    return w

---
# Calculating mesh

In [ ]:
# Beltrami Coefficient of z-dom
def mu(z):
    return 0.0

# Input
N,M = 32,32

# Initialize Z-Vertices and W-Vertices
klist = range(N)
jlist = range(-M,1)
r = math.sqrt(3.)*math.pi/N
z = np.zeros((len(jlist),len(klist)),dtype=np.complex)
w = np.zeros((len(jlist),len(klist)),dtype=np.complex)
for j in jlist:
    for k in klist:
        z[j][k] = r*j + 2*math.pi*1j*(k+(j%2)/2.)/N
w = copy.deepcopy(z)

print("start calculation...c")
start = time.time()

itnam=1
w=iteration(z,w,N,M,itnam)

end = time.time() - start
print ("end: {0}".format(end) + "[sec]")

print("start drawing...",itnam)
start = time.time()
drawmesh(z,N,M)
drawmesh(w,N,M)
end = time.time() - start
print ("end: {0}".format(end) + "[sec]")

AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations